In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import classification_report, accuracy_score
import math

# Loading mast data

We chose to go with the Risoe data, only loading the data that is needed, to reduce memory usage.

In [ ]:
import netCDF4 as nc
import numpy as np
from datetime import datetime, timedelta

file_path_risoe = 'risoe_m_all.nc'
file_paths_borglum = 'borglum_all.nc'

signals_risoe = ['ws77', 'wd77', 'ws125', 'wd125']
signals_borglum = ['ws32', 'wd32']

base_date_borglum = datetime(1997, 12, 11, 16, 5, 0)
base_date_risoe = datetime(1995, 11, 20, 16, 25, 0)

# Get the Risoe dataset:
dataset = nc.Dataset(file_path_risoe, 'r')

# List the variables in the dataset
print("Variables in the netCDF file:")
for var_name in dataset.variables:
    print(var_name)

time_minutes = np.array(dataset.variables['time'])

# Convert time values to timestamp strings
time = []
for minutes in time_minutes:
	time_delta = timedelta(minutes=int(minutes))
	timestamp = base_date_risoe + time_delta
	time.append(timestamp.strftime('%Y-%m-%d %H:%M:%S'))
 
print(f"time:\n {time[:10]} - {time[-1]}")

for signal in signals_risoe:
	values = np.array(dataset.variables[signal])
	print(f'{signal}:\n {values[:10]} - {values[-10:-1]}')

In [ ]:
import netCDF4 as nc
import numpy as np
import pandas as pd

# Make sure to use the correct dataset
file_path = 'Data/Risoe/risoe_m_all.nc'
dataset = nc.Dataset(file_path, 'r')

# Confirm variables are in the dataset
print("Variables in the netCDF file:")
for var_name in dataset.variables:
    print(var_name)

# Ensure the required variables exist before attempting to access them
required_vars = ['time', 'ws77', 'wd77', 'ws125', 'wd125']
for var in required_vars:
    if var not in dataset.variables:
        print(f"Variable {var} is missing in the dataset.")
    else:
        print(f"Variable {var} is available.")

# Convert NetCDF data to pandas DataFrame
df_mast_risoe = pd.DataFrame()
df_mast_risoe['time'] = pd.to_datetime([datetime(1995, 11, 20, 16, 25) + timedelta(minutes=int(m)) for m in dataset.variables['time'][:]])
df_mast_risoe['ws77'] = np.array(dataset.variables['ws77'][:])
df_mast_risoe['wd77'] = np.array(dataset.variables['wd77'][:]%360)
df_mast_risoe['ws125'] = np.array(dataset.variables['ws125'][:])
df_mast_risoe['wd125'] = np.array(dataset.variables['wd125'][:]%360)

# Setting the 'time' column as the index
df_mast_risoe.set_index('time', inplace=True)

# Display the last 25 rows to confirm data is loaded correctly
print(df_mast_risoe.tail(25))


making windroses to determine the wind speeds from directions. count based

In [ ]:
from windrose import WindroseAxes
import matplotlib.pyplot as plt

ws_risoe_77 = df_mast_risoe['ws77'].to_numpy()
wd_risoe_77 = df_mast_risoe['wd77'].to_numpy()

# Remove NaN values from both arrays
valid_indices = ~np.isnan(ws_risoe_77) & ~np.isnan(wd_risoe_77)
ws_risoe_77_clean = ws_risoe_77[valid_indices]
wd_risoe_77_clean = wd_risoe_77[valid_indices]

# Plot wind rose
ax = WindroseAxes.from_ax()
ax.bar(wd_risoe_77_clean, ws_risoe_77_clean)
ax.set_legend()

plt.show()

In [ ]:
# Plot wind rose
ax = WindroseAxes.from_ax()
ax.bar(wd_risoe_77_clean, ws_risoe_77_clean, normed=True)
ax.set_legend()

# Format radius axis to percentages
fmt = '%.0f%%'
yticks = mtick.FormatStrFormatter(fmt)
ax.yaxis.set_major_formatter(yticks)

plt.show()

Converting the data to a Pandas DataFrame, to make it easier to work with.
Converting time to datetime objects, and setting it as the index.

In [ ]:
df_mast_risoe = pd.DataFrame()
df_mast_risoe['time'] = pd.to_datetime([datetime(1995, 11, 20, 16, 25) + timedelta(minutes=int(m)) for m in dataset.variables['time'][:]])
df_mast_risoe['ws77'] = np.array(dataset.variables['ws77'][:])
df_mast_risoe['wd77'] = np.array(dataset.variables['wd77'][:])
df_mast_risoe['ws125'] = np.array(dataset.variables['ws125'][:])
df_mast_risoe['wd125'] = np.array(dataset.variables['wd125'][:])

df_mast_risoe.set_index('time', inplace=True)

## Handleing missing values

Plotting the data so se where data is mussing

In [ ]:
sns.set()
df_mast_risoe.plot(subplots=True, layout=(4,1), figsize=(12,12), sharex=False, sharey=False)